Start with differences. Start from the standard decomposition GUI. 

Also the differences can be generated as per:
```residual_surfaces = [s - u for s, u in zip(self.surfaces, upsampled_surfaces)]```

We can use the old pyramid code.

In [1]:
import numpy as np
from src.var_processor.surface_stage import decompose

def create_decom_pyramid(image):
    """Create a decomposition image pyramid from the image."""
    # Convert to 16-bit signed to allow decomposition
    image = image.astype(np.int16)
    rows, cols = image.shape
    scales = min(rows, cols).bit_length()-1
    # Create list to hold scales
    pyramid = []
    current_image = image
    for _ in range(scales):
        # Decompose
        surfaces = decompose(current_image)
        # Save surfaces in pyramid
        pyramid.append(surfaces)
        current_image = surfaces[0]
    return pyramid

In [2]:
import cv2
from src.visualisers.camera_gui import DecomposeFrame, BasicCameraGUI

class DeComGUI(BasicCameraGUI):
    """GUI to look at decompositions."""

    def __init__(self, src=0):
        # Call parent init
        super().__init__(src)

        # Determine decomposition stages for now
        (readsuccessful, frame) = self.cam.read()
        image = frame[:, :, 0]
        rows, cols = image.shape
        self.num_of_stages = min(rows, cols).bit_length()-1

        # Create a frame for each stage and pack vertically
        self.frames = [
            DecomposeFrame(self.window, width=128, height=128)
            for _ in range(self.num_of_stages)]

    def update_image(self):
        # Get frame
        (readsuccessful, frame) = self.cam.read()
        image = frame[:, :, 0]
        # Reduce to a power of 2
        image = reduce_to_2(image)
        # Generate pyramid
        pyramid = create_decom_pyramid(image)
        # Display
        for frame, surfaces in zip(self.frames, pyramid):
            images = [surfaces[0].astype(np.uint8)]
            images += [
                cv2.applyColorMap((i + 128).astype(np.uint8), cv2.COLORMAP_JET) for i in surfaces[1:]
            ]
            frame.update(images)
        return image


In [3]:
decom = DeComGUI()
# decom.run()

Note that frame rate is down to 22 for this. Also low-resolution images aren't displayed very well.

In [4]:
self = decom
(readsuccessful, frame) = self.cam.read()
image = frame[:, :, 0]
# Generate pyramid
pyramid = create_decom_pyramid(image)

In [5]:
for surfaces in pyramid:
    print([s.shape for s in surfaces])

[(240, 320), (240, 320), (240, 320), (240, 320)]
[(120, 160), (120, 160), (120, 160), (120, 160)]
[(60, 80), (60, 80), (60, 80), (60, 80)]
[(30, 40), (30, 40), (30, 40), (30, 40)]
[(15, 20), (15, 20), (15, 20), (15, 20)]
[(7, 10), (7, 10), (7, 10), (7, 10)]
[(3, 5), (3, 5), (3, 5), (3, 5)]
[(1, 2), (1, 2), (1, 2), (1, 2)]


We need reduce_to_2 function to allow upsampling and downsampling size matches.

In [6]:
from src.sources.video_helper import reduce_to_2

In [7]:
self = decom
(readsuccessful, frame) = self.cam.read()
image = frame[:, :, 0]
# Reduce to a power of 2
reduced = reduce_to_2(image)
# Generate pyramid
pyramid = create_decom_pyramid(reduced)
for surfaces in pyramid:
    print([s.shape for s in surfaces])

[(128, 128), (128, 128), (128, 128), (128, 128)]
[(64, 64), (64, 64), (64, 64), (64, 64)]
[(32, 32), (32, 32), (32, 32), (32, 32)]
[(16, 16), (16, 16), (16, 16), (16, 16)]
[(8, 8), (8, 8), (8, 8), (8, 8)]
[(4, 4), (4, 4), (4, 4), (4, 4)]
[(2, 2), (2, 2), (2, 2), (2, 2)]
[(1, 1), (1, 1), (1, 1), (1, 1)]


Actually this loses a lot of the detail 240 > 128. We coul;d actually just pad using mirrored or continue. But this isn't as much of a problem once we split the visual field and use the polar mapping.

In [8]:
decom.run()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/ben/anaconda3/envs/ml_platform_3_7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/ben/anaconda3/envs/ml_platform_3_7/lib/python3.7/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/ben/Projects/var_processor/src/visualisers/camera_gui.py", line 89, in <lambda>
    self.window.after(20, func=lambda: self.update_all())
  File "/home/ben/Projects/var_processor/src/visualisers/camera_gui.py", line 88, in update_all
    self.update_fps()
  File "/home/ben/Projects/var_processor/src/visualisers/camera_gui.py", line 82, in update_fps
    self.fps_label.configure(text=f'FPS: {fps}')
  File "/home/ben/anaconda3/envs/ml_platform_3_7/lib/python3.7/tkinter/__init__.py", line 1485, in configure
    return self._configure('configure', cnf, kw)
  File "/home/ben/anaconda3/envs/ml_platform_3_7/lib/python3.7/tkinter/__init__.py", line 1476, in _confi

In [ ]:
def upsample(pyramid):
    """Upsample pyramid."""
    # Add the 1x1 pixel reconstruction to the rebuilt list
    upsampled = [pyramid[-1]]
    # Check for 1D pyramid
    one_d = True if pyramid[0].shape[1] == 1 else False
    # Interate through the images and add reconstructed versions
    for image in reversed(pyramid[1:]):
        rows, cols = image.shape
        new_rows = rows*2
        # Set cols to 1 if one_d else double size
        new_cols = 1 if one_d else cols*2
        upsampled.append(cv2.pyrUp(image, dstsize=(new_cols, new_rows)))
    return upsampled

In [ ]:
class DiffGUI(BasicCameraGUI):
    """GUI to look at decompositions."""

    def __init__(self, src=0):
        # Call parent init
        super().__init__(src)

        # Determine decomposition stages for now
        (readsuccessful, frame) = self.cam.read()
        image = frame[:, :, 0]
        rows, cols = image.shape
        self.num_of_stages = min(rows, cols).bit_length()-1

        # Create a frame for each stage and pack vertically
        self.frames = [
            DecomposeFrame(self.window, width=128, height=128)
            for _ in range(self.num_of_stages)]

    def update_image(self):
        # Get frame
        (readsuccessful, frame) = self.cam.read()
        image = frame[:, :, 0]
        # Generate pyramid
        pyramid = create_decom_pyramid(image)
        
        # Display
        for frame, surfaces in zip(self.frames, pyramid):
            images = [surfaces[0].astype(np.uint8)]
            images += [
                cv2.applyColorMap((i + 128).astype(np.uint8), cv2.COLORMAP_JET) for i in surfaces[1:]
            ]
            frame.update(images)
        return image